In [10]:
"""
C10でFine-tuningしたViTモデルのフォワードパスを実行する
"""

import os, sys, math
from tqdm.notebook import tqdm
sys.path.append("../src")
import numpy as np
import torch
from datasets import load_from_disk
from transformers import DefaultDataCollator, ViTForImageClassification, TrainingArguments, Trainer
from utils.helper import get_device
from utils.vit_util import processor, transforms, compute_metrics

In [4]:
# dsのsaveとload
from datasets import load_from_disk
cifar10 = load_from_disk("../dataset/c10")
cifar10

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 10000
    })
})

In [5]:
# デバイス (cuda, or cpu) の取得
device = get_device()
# 読み込まれた時にリアルタイムで前処理を適用するようにする
cifar10_preprocessed = cifar10.with_transform(transforms)
# バッチごとの処理のためのdata_collator
data_collator = DefaultDataCollator()
# ラベルを示す文字列のlist
labels = cifar10_preprocessed["train"].features["label"].names
# pretrained modelのロード
pretrained_dir = "/src/src/out_vit_c10"
model = ViTForImageClassification.from_pretrained(pretrained_dir).to(device)
model.eval()
# 学習時の設定をロード
training_args = torch.load(os.path.join(pretrained_dir, "training_args.bin"))
# Trainerオブジェクトの作成
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=cifar10_preprocessed["train"],
    eval_dataset=cifar10_preprocessed["test"],
    tokenizer=processor,
)

Device: cuda


In [16]:
for entry_dic in tqdm(cifar10_preprocessed["train"].iter(batch_size=32), total=len(cifar10_preprocessed["train"])//32+1):
    x, y = entry_dic["pixel_values"].to(device), entry_dic["labels"]
    output = model.forward(x, output_hidden_states=True)
    print(len(output.hidden_states))

  0%|          | 0/1563 [00:00<?, ?it/s]

13
13


KeyboardInterrupt: 

In [9]:
mf = model.forward(dum_input, output_hidden_states=True, output_attentions=True, output_intermediate_states=True)
print(mf.keys())
# 後ろの3つのキーのタプルの長さを取得
print([len(mf[k]) for k in list(mf.keys())[-3:]])

odict_keys(['logits', 'hidden_states', 'attentions', 'intermediate_states'])
[13, 12, 12]


In [5]:
mf.logits.shape

torch.Size([5, 10])

In [6]:
print(len(mf.hidden_states))
for i, hs in enumerate(mf.hidden_states):
    print(f"mf.hidden_states[{i}].shape = {hs.shape}")
    print(f"mf.hidden_states[{i}][:, 0, :].shape = {hs[:, 0, :].shape}")

13
mf.hidden_states[0].shape = torch.Size([5, 197, 768])
mf.hidden_states[0][:, 0, :].shape = torch.Size([5, 768])
mf.hidden_states[1].shape = torch.Size([5, 197, 768])
mf.hidden_states[1][:, 0, :].shape = torch.Size([5, 768])
mf.hidden_states[2].shape = torch.Size([5, 197, 768])
mf.hidden_states[2][:, 0, :].shape = torch.Size([5, 768])
mf.hidden_states[3].shape = torch.Size([5, 197, 768])
mf.hidden_states[3][:, 0, :].shape = torch.Size([5, 768])
mf.hidden_states[4].shape = torch.Size([5, 197, 768])
mf.hidden_states[4][:, 0, :].shape = torch.Size([5, 768])
mf.hidden_states[5].shape = torch.Size([5, 197, 768])
mf.hidden_states[5][:, 0, :].shape = torch.Size([5, 768])
mf.hidden_states[6].shape = torch.Size([5, 197, 768])
mf.hidden_states[6][:, 0, :].shape = torch.Size([5, 768])
mf.hidden_states[7].shape = torch.Size([5, 197, 768])
mf.hidden_states[7][:, 0, :].shape = torch.Size([5, 768])
mf.hidden_states[8].shape = torch.Size([5, 197, 768])
mf.hidden_states[8][:, 0, :].shape = torch.Size

In [10]:
print(len(mf.attentions))
for i, attn in enumerate(mf.attentions):
    print(f"mf.attentions[{i}].shape = {attn.shape}")

12
mf.attentions[0].shape = torch.Size([5, 12, 197, 197])
mf.attentions[1].shape = torch.Size([5, 12, 197, 197])
mf.attentions[2].shape = torch.Size([5, 12, 197, 197])
mf.attentions[3].shape = torch.Size([5, 12, 197, 197])
mf.attentions[4].shape = torch.Size([5, 12, 197, 197])
mf.attentions[5].shape = torch.Size([5, 12, 197, 197])
mf.attentions[6].shape = torch.Size([5, 12, 197, 197])
mf.attentions[7].shape = torch.Size([5, 12, 197, 197])
mf.attentions[8].shape = torch.Size([5, 12, 197, 197])
mf.attentions[9].shape = torch.Size([5, 12, 197, 197])
mf.attentions[10].shape = torch.Size([5, 12, 197, 197])
mf.attentions[11].shape = torch.Size([5, 12, 197, 197])


In [11]:
print(len(mf.intermediate_states))
for i, med in enumerate(mf.intermediate_states):
    print(f"mf.intermediate_states[{i}].shape = {med.shape}")

12
mf.intermediate_states[0].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[1].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[2].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[3].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[4].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[5].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[6].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[7].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[8].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[9].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[10].shape = torch.Size([5, 197, 3072])
mf.intermediate_states[11].shape = torch.Size([5, 197, 3072])
